# CSC 480-F25 Lab 5: Games as Adversarial Search (Scrabble Agent)

### Authors:
***Arnav Bhola, Pranav Krishna***

California Polytechnic State University, San Luis Obispo;
Computer Science & Software Engineering Department

### Overview
This lab covers **Adversarial Search**, beginning with classical Minimax and Alpha-Beta pruning, then transitioning to **Monte Carlo Tree Search (MCTS)** to handle the complexity and imperfect information of Scrabble. We apply the **agentic design approach** (from Lab 2 and the heuristic integration of Lab 4) to construct a sophisticated state evaluation function that guides the MCTS playouts.

---

## Environment Setup and Imports

In [1]:
# Reusing package install from L4
%pip install "autogen-core" "autogen-agentchat" "autogen-ext[openai,azure]"


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import asyncio
import time
import random
from typing import List, Tuple, Dict, Optional

# Import AutoGen classes
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

# Import core game utilities from scrabble_utils.py (must be in the same directory)
from L5_utils import ScrabbleState, AlphaBetaMinimax, MonteCarlo, Move 

### First take a look at the Scrabble Implementation

The following code randomly selects moves for both players in the Scrabble game until the game terminates.
It should give you a sense of what the API looks like.

In [3]:
# Initialize the game state
game_state = ScrabbleState.create_new_game()
print("Initial Game State:")
print(game_state)

while not game_state.is_terminal():
    current_player = game_state.current_player
    legal_moves = game_state.get_legal_moves(current_player)

    # Prefer non-pass moves if available
    non_pass_moves = [move for move in legal_moves if not move.is_pass]
    if non_pass_moves:
        chosen_move = random.choice(non_pass_moves)
    else:
        chosen_move = legal_moves[0]  # Must pass if no other
    
    game_state = game_state.apply_move(chosen_move)
    print(f"\nPlayer {current_player} plays: {chosen_move}")
    print(game_state)

Initial Game State:
SCRABBLE GAME STATE

Current Player: Player 1
Scores: Player 1: 0, Player 2: 0
Consecutive Passes: 0

Player 1 Rack: Z Y G E H B O (7 tiles)
Player 2 Rack: I I N H J F B (7 tiles)

Tiles Remaining in Pool: 86

Board:
Legend: · = empty, ² = 2x letter, ³ = 3x letter, * = 2x word, # = 3x word
     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
   +---------------------------------------------+
 0 | ·  ²  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ²  · |
 1 | ²  ·  ·  ·  ·  ³  ·  ·  ·  ³  ·  ·  ·  ·  ² |
 2 | ·  ·  *  ·  ·  ·  ²  ·  ²  ·  ·  ·  *  ·  · |
 3 | ·  ·  ·  #  ·  ·  ·  ²  ·  ·  ·  #  ·  ·  · |
 4 | ·  ·  ·  ·  *  ·  ·  ·  ·  ·  *  ·  ·  ·  · |
 5 | ·  ³  ·  ·  ·  ³  ·  ·  ·  ³  ·  ·  ·  ³  · |
 6 | ·  ·  ²  ·  ·  ·  ²  ·  ²  ·  ·  ·  ²  ·  · |
 7 | ·  ·  ·  ²  ·  ·  ·  *  ·  ·  ·  ²  ·  ·  · |
 8 | ·  ·  ²  ·  ·  ·  ²  ·  ²  ·  ·  ·  ²  ·  · |
 9 | ·  ³  ·  ·  ·  ³  ·  ·  ·  ³  ·  ·  ·  ³  · |
10 | ·  ·  ·  ·  *  ·  ·  ·  ·  ·  *  ·  ·  ·  · |
11 | ·  ·  ·  #  ·  ·  ·  ² 

### Azure OpenAI Configuration

In [4]:
from pathlib import Path
from dotenv import load_dotenv

In [5]:
cwd = Path.cwd()
env_path = cwd.parent / ".env"
loaded = load_dotenv(env_path)

In [6]:
# Reusing configuration from L4.ipynb
# Configure your Azure OpenAI client
azure_deployment = os.getenv("AZURE_DEPLOYMENT_NAME")
api_version = "2024-12-01-preview" 
azure_endpoint = os.getenv("AZURE_ENDPOINT")

# Ensure your API key is set as an environment variable
api_key = os.getenv("AZURE_SUBSCRIPTION_KEY")

client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_deployment,
    model="gpt-5-mini", # Using the default model name referenced in previous labs
    api_version=api_version,
    azure_endpoint=azure_endpoint,
    api_key=api_key,
)

---

## Part 1 & 2: Classical Search Limitations

A small note: Minimax and MCTS expect that the problem space is zero-sum. Scrabble is not inherently zero-sum with the typical method scores are calculated. However, we make it zero-sum by using the difference between the two players' scores as the metric to optimize for.

### 1. Written Analysis: Why Minimax Fails Scrabble (Part 2)

**Task:** Using the concepts of **State Space Size, Branching Factor, and Imperfect Information** (hidden opponent tiles), justify why a naive application of Minimax with Alpha-Beta pruning is impractical for Scrabble.

Scrabble presents several fundamental challenges that make classical Minimax search impractical. At any given turn, a player typically has hundreds of possible legal moves due to the 7-tile rack and 15×15 board, creating an enormous branching factor that leads to exponential node explosion even with Alpha-Beta pruning. The state space is astronomically large with 100 tiles and countless possible board configurations, far exceeding games like Chess. Most critically, Scrabble involves imperfect information since players cannot see their opponent's tiles, which violates Minimax's assumption of perfect information and forces the algorithm to either make unrealistic assumptions or search over all possible opponent tile combinations. A typical Scrabble game lasts 15 to 25 turns per player, and searching even 3 to 4 moves ahead with a branching factor of 200 would require evaluating millions of positions, making it computationally prohibitive. These factors collectively necessitate alternative approaches like Monte Carlo Tree Search that can handle large state spaces and imperfect information through sampling rather than exhaustive search.

### 2. Demonstration: Minimax Performance Limits

**Task:** Run the `AlphaBetaMinimax` on a simplified Scrabble state. Observe the exploration time and the number of nodes explored, particularly when increasing the `max_depth` to demonstrate the complexity explosion.

In [7]:
# Initialize a simple Scrabble state
initial_state = ScrabbleState.create_new_game()
print(f"Initial state created. Player 1 Rack: {initial_state.racks}")

# Experiment 1: Shallow Search (Depth 1)
print("\n--- Running Minimax (Depth 1) ---")
minimax_d1 = AlphaBetaMinimax(max_depth=1)
best_move_d1 = minimax_d1.find_best_move(initial_state)
print(f"Depth 1 suggests: {best_move_d1}")

# Experiment 2: Deeper Search (Depth 3)
print("\n--- Running Minimax (Depth 3) ---")
minimax_d3 = AlphaBetaMinimax(max_depth=3)

best_move_d3 = minimax_d3.find_best_move(initial_state)
print(f"Depth 3 suggests: {best_move_d3}")

Initial state created. Player 1 Rack: {1: ['A', 'Q', 'N', 'E', '_', 'I', 'P'], 2: ['E', '_', 'R', 'X', 'M', 'S', 'R']}

--- Running Minimax (Depth 1) ---
Minimax search completed in: 0.0003s. Nodes explored: 25. Best move value: 30.00
Depth 1 suggests: Move(Word='IPAQ', Score=30, Tile Positions=[(7, 4), (7, 5), (7, 6), (7, 7)])

--- Running Minimax (Depth 3) ---
Minimax search completed in: 1.4313s. Nodes explored: 158. Best move value: 47.00
Depth 3 suggests: Move(Word='PAIN', Score=12, Tile Positions=[(6, 7), (7, 7), (8, 7), (9, 7)])


---

## Part 3: Adversarial Monte Carlo Agent

Now check out MCTS with the placeholder heuristic (which currently uses the score + noise).

In [8]:
async def basic_scrabble_heuristic(state: ScrabbleState, player: int) -> float:
    # Simple heuristic: score + small random noise
    base_score = state.scores[player]
    noise = random.uniform(-0.1, 0.1)  # Small noise to break ties
    return base_score + noise


Here we're simulating a game between two players. One which uses MCTS to choose it's moves (Player 1) and the other which chooses at random (Player 2).

In [9]:
async def run_mcts_match(num_playouts: int, heuristic_fn, num_turns: int = 5):
    """Initializes and runs the Monte Carlo Agent against a simple opponent."""

    # MCTS Agent (Player 1) uses the given scrabble_heuristic
    mcts = MonteCarlo(num_playouts=num_playouts, heuristic_fn=heuristic_fn)

    current_state = ScrabbleState.create_new_game()

    print(
        f"Starting Scrabble game simulation for {num_turns} turns. MCTS Agent (P1) uses {int(num_playouts)} playouts."
    )

    for turn in range(num_turns):
        if current_state.is_terminal():
            break

        player_id = current_state.current_player

        if player_id == 1:
            # Player 1 uses MCTS
            print(f"\n--- Turn {turn+1}: Player 1 (MCTS) ---")
            move = await mcts.find_best_move(current_state)

        else:
            # Player 2 makes moves randomly (baseline)
            print(f"\n--- Turn {turn+1}: Player 2 (Random Baseline) ---")
            possible_moves = current_state.get_legal_moves(player_id)
            move = random.choice(possible_moves)

        current_state = current_state.apply_move(move)
        print(f"Player {player_id} plays: {move}")
        print(current_state)


# Task: Run a baseline MCTS match
await run_mcts_match(num_playouts=1e5, heuristic_fn=basic_scrabble_heuristic)

Starting Scrabble game simulation for 5 turns. MCTS Agent (P1) uses 100000 playouts.

--- Turn 1: Player 1 (MCTS) ---
Starting MCTS: 1176 playouts per move (for 85 possible moves)...
Player 1 plays: Move(Word='HIRED', Score=26, Tile Positions=[(7, 3), (7, 4), (7, 5), (7, 6), (7, 7)])
SCRABBLE GAME STATE

Current Player: Player 2
Scores: Player 1: 26, Player 2: 0
Consecutive Passes: 0

Player 1 Rack: X I G T R O U (7 tiles)
Player 2 Rack: B O F E T A T (7 tiles)

Tiles Remaining in Pool: 81

Board:
Legend: · = empty, ² = 2x letter, ³ = 3x letter, * = 2x word, # = 3x word
     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
   +---------------------------------------------+
 0 | ·  ²  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ²  · |
 1 | ²  ·  ·  ·  ·  ³  ·  ·  ·  ³  ·  ·  ·  ·  ² |
 2 | ·  ·  *  ·  ·  ·  ²  ·  ²  ·  ·  ·  *  ·  · |
 3 | ·  ·  ·  #  ·  ·  ·  ²  ·  ·  ·  #  ·  ·  · |
 4 | ·  ·  ·  ·  *  ·  ·  ·  ·  ·  *  ·  ·  ·  · |
 5 | ·  ³  ·  ·  ·  ³  ·  ·  ·  ³  ·  ·  ·  ³  · |
 6 | ·  ·  ²  ·  

---

## Part 4: Agentic Heuristic Design and Impact


Implement the `collaboratie_heuristic` function using AutoGen agent collaboration.

Note: Like previous labs that use an agentic heuristic function, this will take a long time to run. An agentic system for this purpose is impractical due to the overhead of making many LLM calls.

In [ ]:
async def setup_agentic_heuristic_system(client):
    """Instantiate heuristic agents for Scrabble state evaluation."""

    # 1. Define Heuristic Agents (using roles defined in Part 4)
    # The system prompts MUST instruct the agents to output a clear numeric score.
    score_estimator = AssistantAgent(
        name="Score_Estimator",
        system_message="You calculate the potential points generated by a proposed Scrabble move and potential follow-up points. Provide rationale and end with 'SCORE_EST: <float>'.",
        model_client=client,
    )
    tile_quality_agent = AssistantAgent(
        name="Tile_Quality_Agent",
        system_message="You analyze the current player's remaining tile rack quality (diversity, future bingo potential) and assign a score. Provide rationale and end with 'SCORE_TILE: <float>'.",
        model_client=client,
    )
    opponent_risk_agent = AssistantAgent(
        name="Opponent_Risk_Agent",
        system_message="You assess the strategic risk/vulnerability created for the opponent by the current board state (e.g., exposed triple word scores). Provide rationale and end with 'SCORE_RISK: <float>'.",
        model_client=client,
    )

    # 2. Define the Aggregator Agent (Orchestrator for the heuristic calculation)
    mcts_aggregator = AssistantAgent(
        name="MCTS_Aggregator",
        system_message="You receive analyses from the Score, Tile, and Risk agents. Combine their numeric scores using a defined weighting (e.g., Score*0.5 + Tile*0.3 + Risk*0.2) and return a single composite score. Always end your final response with 'FINAL_SCORE: <float>'.",
        model_client=client,
    )

    return [score_estimator, tile_quality_agent, opponent_risk_agent], mcts_aggregator

async def collaborative_heuristic(state: ScrabbleState, player: int) -> float:
    heuristic_agents, aggregator = await setup_agentic_heuristic_system(client)

    task = f"Evaluate the strategic value of this Scrabble State for the current player using collaboration:\n{state}"

    groupchat = RoundRobinGroupChat(
        heuristic_agents + [aggregator],
        max_turns=1,
    )

    result = await groupchat.run(task=task)

    # Parse the final output from the aggregator
    final_output = result.messages[-1].content
    try:
        score_str = final_output.split("FINAL_SCORE:")[-1].strip()
        return float(score_str)
    except:
        print("Warning: Failed to parse final score. Returning baseline.")
        return state.get_utility(state.current_player)

# To run the experiment: replace the placeholder function in MCTS Agent
await run_mcts_match(num_playouts=1, heuristic_fn=collaborative_heuristic)

Starting Scrabble game simulation for 5 turns. MCTS Agent (P1) uses 1 playouts.

--- Turn 1: Player 1 (MCTS) ---
Starting MCTS: 1 playouts per move (for 201 possible moves)...


---

## Reflection & Analysis

##### What worked well in the Agentic Heuristic System?

_(Reflect on the clarity of A2A communication, the effectiveness of the Aggregator, etc.)_

##### What struggled?

_(Note where agents needed multiple turns, produced inconsistent scores, or struggled to integrate the information.)_

##### Agentic vs. Classical Heuristics

_(Compare the complexity and potential strength of the composite heuristic created via agent collaboration versus a monolithic Python function, addressing modularity and extensibility.)_

The agentic heuristic system demonstrates significantly greater modularity and extensibility compared to a monolithic Python function, though at a substantial computational cost. In the classical approach, all heuristic logic is embedded within a single function where scoring considerations like tile quality, board positioning, and opponent threats are manually weighted and combined by the programmer. This requires deep domain expertise and makes modifications difficult since changing one aspect may have unintended effects on others. In contrast, the agentic system distributes evaluation responsibilities across specialized agents, each focusing on a distinct strategic dimension such as score estimation, tile rack quality, or opponent risk assessment. This separation of concerns allows each agent to develop nuanced reasoning about its specific domain and communicate findings in natural language before the aggregator synthesizes them into a final score. The modularity means new strategic considerations can be added by simply introducing new agents without rewriting existing logic, and individual agents can be refined independently. However, the agentic approach is dramatically slower due to multiple LLM calls per evaluation, making it impractical for real-time game play where thousands of heuristic evaluations are needed. The classical heuristic executes in microseconds while the agentic version may take seconds per evaluation. Ultimately, the agentic system offers superior flexibility and interpretability with explicit reasoning traces, but the classical approach remains necessary for computational efficiency in actual gameplay scenarios like MCTS where speed is critical.